In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import datetime

# from matplotlib import pyplot as plt
# import seaborn as sns
# %matplotlib inline

In [7]:
#Read in files and put into dataframe
file_1 = pd.read_csv(str(input('File_1 Name: ')), skiprows=12, encoding='latin1') #Chow Cohort- Ghrelin pt. 1.csv
file_2 = pd.read_csv(str(input('File_2 Name: ')), skiprows=12, encoding='latin1') #Chow Cohort- Ghrelin pt. 2.csv

#Store and drop first row containing measurement units
measurement_units_dict = dict(zip(
    file_1.drop(['Date Time', 'Animal No.', 'Box', 'Unnamed: 25'], axis=1).fillna('[RER]').columns, 
    file_1.drop(['Date Time', 'Animal No.', 'Box', 'Unnamed: 25'], axis=1).fillna('[RER]').iloc[0].values)
                             )
file_1 = file_1.drop(index=0)
file_1.drop('Unnamed: 25', axis=1, inplace=True)
file_1 = file_1.reset_index(drop=True)
file_2 = file_2.drop(index=0)
file_2.drop('Unnamed: 25', axis=1, inplace=True)
file_2 = file_2.reset_index(drop=True)

#Convert Date Time to datetime and other columns to numeric (RER won't convert if an animal is missing...will convert later)
file_1 = file_1.apply(pd.to_numeric, errors='ignore')
file_1['Date Time'] = file_1['Date Time'].apply(pd.to_datetime)

file_2 = file_2.apply(pd.to_numeric, errors='ignore')
file_2['Date Time'] = file_2['Date Time'].apply(pd.to_datetime)

File_1 Name:  Chow Cohort- Ghrelin pt. 1.csv
File_2 Name:  Chow Cohort- Ghrelin pt. 2.csv


In [8]:
# Create new columns for date/hour/minute and then drop the earlier starting df rows until start hour is the same
file_1['date'] = file_1['Date Time'].dt.date
file_1['hour'] = file_1['Date Time'].dt.hour
file_1['minute'] = file_1['Date Time'].dt.minute

file_2['date'] = file_2['Date Time'].dt.date
file_2['hour'] = file_2['Date Time'].dt.hour
file_2['minute'] = file_2['Date Time'].dt.minute

# Drop the first time point for each animal until the HOURS are equal
time_start_file_1_hour = np.arange(0, len(file_1['hour']), (len(file_1['hour'])/8), dtype=int)
time_start_file_2_hour = np.arange(0, len(file_2['hour']), (len(file_2['hour'])/8), dtype=int)

while int(abs(file_2['hour'].iloc[0] - file_1['hour'].iloc[0])) != 0:
    time_start_file_1_hour = np.arange(0, len(file_1['hour']), (len(file_1['hour'])/8))
    time_start_file_2_hour = np.arange(0, len(file_2['hour']), (len(file_2['hour'])/8))
    file_1 = file_1.drop(index=time_start_file_1_hour)
    file_1 = file_1.reset_index(drop=True)
        
# Drop the first time point for each animal until the MINUTES are <30 min (readings every 27 min)
time_start_file_1_minute = np.arange(0, len(file_1['minute']), (len(file_1['minute'])/8), dtype=int)
time_start_file_2_minute = np.arange(0, len(file_2['minute']), (len(file_2['minute'])/8), dtype=int)

while int(abs(file_2['minute'].iloc[0] - file_1['minute'].iloc[0])) > 30:
    time_start_file_1_minute = np.arange(0, len(file_1['minute']), (len(file_1['minute'])/8))
    time_start_file_2_minute = np.arange(0, len(file_2['minute']), (len(file_2['minute'])/8))
    file_1 = file_1.drop(index=time_start_file_1_minute)
    file_1 = file_1.reset_index(drop=True)
    
# Reduces number of time points for each animal in longer df to match animal value_counts in shorter df
time_matched_df = pd.DataFrame()
if len(file_2) > len(file_1):
    for i in file_2['Animal No.'].unique():
        time_matched_df = time_matched_df.append(file_2[file_2['Animal No.'] == i].iloc[:np.int(len(file_1)/8)])
        time_matched_df = time_matched_df.reset_index(drop=True)
    file_2 = time_matched_df

if len(file_2) < len(file_1):
    for i in file_1['Animal No.'].unique():
        time_matched_df = time_matched_df.append(file_1[file_1['Animal No.'] == i].iloc[:np.int(len(file_2)/8)])
        time_matched_df = time_matched_df.reset_index(drop=True)
    file_1 = time_matched_df

In [9]:
#Make Date Time for both files the same and concat files into single dataframe
file_2['Date Time'] = file_1['Date Time']
df = pd.concat([file_1, file_2]).reset_index(drop=True)

In [10]:
#Create lists for Cre+ and WT animal numbers (or any two groups)
group_1_list = input('Group_1 Animals (#, #, #...): ') #476, 481, 478, 487, 484, 491, 493
group_1_list = [int(s) for s in group_1_list.split(', ') if s.isdigit()]

group_2_list = input('Group_2 Animals (#, #, #...): ') #480, 477, 479, 482, 483, 489, 486
group_2_list = [int(s) for s in group_2_list.split(', ') if s.isdigit()]

#Create genotype identifiers for each group
group_1_genotype = input('Group_1 Genotype: ') #cre
group_2_genotype = input('Group_1 Genotype: ') #WT

Group_1 Animals (#, #, #...):  76, 481, 478, 487, 484, 491, 493
Group_2 Animals (#, #, #...):  480, 477, 479, 482, 483, 489, 486
Group_1 Genotype:  cre
Group_1 Genotype:  WT


In [11]:
#Create dict of animal id's and genotye to map
genotype_map_dict = dict(zip(group_1_list, [group_1_genotype for i in group_1_list]))
genotype_map_dict.update(dict(zip(group_2_list, [group_2_genotype for i in group_2_list])))

#Create new column for genotype and map each genotype according to animal id and corresponding group
df['Genotype'] = df['Animal No.'].map(genotype_map_dict)

#Drop any rows not containing an animal and convert RER to numeric
df = df.dropna().reset_index(drop=True)
df = df.apply(pd.to_numeric, errors='ignore')
df['Date Time'] = df['Date Time'].apply(pd.to_datetime)

In [12]:
#Drop unimportant columns and create .csv of combined, processed file
df = df.drop(columns=['date', 'hour', 'minute'])
#combined_processed_file = str(input('Output File Name (*.csv): '))
#df.to_csv(path_or_buf=combined_processed_file)

In [13]:
# plt.hist(df[df['Genotype'] == group_1_genotype]['RER'], color='r', alpha=0.5, bins=30, label=group_1_genotype)
# plt.hist(df[df['Genotype'] == group_2_genotype]['RER'], color='g', alpha=0.5, bins=30, label=group_2_genotype)
# plt.legend(loc='upper right')
# plt.title('RER')
# plt.ylabel(measurement_units_dict['RER'])
# plt.show()

In [14]:
# STATISTICS
# https://pythonfordatascience.org/anova-2-way-n-way/
# https://www.marsja.se/repeated-measures-anova-in-python-using-statsmodels/
# https://www.marsja.se/two-way-anova-repeated-measures-using-python/
# http://www.statsmodels.org/stable/mixed_linear.html
# import researchpy as rp
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [35]:
# Create column with Date Time point number as integer
time_num_dict = dict(zip(df['Date Time'].astype('object').unique(), range(0, np.array(len(df['Date Time'].unique())))))
df['time_num'] = df['Date Time'].map(time_num_dict)
df['animal'] = df['Animal No.']

In [36]:
# Rename columns so they're easier to work with in statsmodels
df = df.rename({'VO2(1)': 'VO2_1', 'VO2(2)': 'VO2_2', 'VO2(3)': 'VO2_3',
               'VCO2(1)': 'VCO2_1', 'VCO2(2)': 'VCO2_2', 'VCO2(3)': 'VCO2_3',
               'H(1)': 'H_1', 'H(2)': 'H_2', 'H(3)': 'H_3', 'XT+YT': 'MVMT_CTS'}, axis=1)
df.columns[7:-3]

Index(['Temp', 'O2', 'CO2', 'dO2', 'dCO2', 'VO2_1', 'VO2_2', 'VO2_3', 'VCO2_1',
       'VCO2_2', 'VCO2_3', 'RER', 'H_1', 'H_2', 'H_3', 'MVMT_CTS', 'Feed',
       'Weight'],
      dtype='object')

In [46]:
# Create dict to store p-values for each column
measurement_pvalue_dict = dict()
for i in df.columns[7:-3]:
    mixed_lm = smf.mixedlm(f"{str(i)} ~ time_num + Genotype", df, groups=df["animal"])
    mixed_lm_fit = mixed_lm.fit()
    mixed_lm_multitest = statsmodels.stats.multitest.multipletests(mixed_lm_fit.pvalues)
	
	# Update dict to include any significant measurements and associated P-value
    measurement_pvalue_dict[i] = str(mixed_lm_multitest[1][1])
	
# Holm-Sidak adjusted P-value
hs_adj_pvalue = mixed_lm_multitest[-2]

C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the paramete

In [47]:
# Create dataframe to store measurement, associated p-value, and any significantly different time points (Holm-Sidak adjusted)
# make df, then for every iteration append df column rows for measurement, anova p-value, and any timepoints that are sig (may do separate)

measurement = list()
measurement_p_val = list()

for i in df.columns[7:-3]:
    mixed_lm = smf.mixedlm(f"{str(i)} ~ time_num + Genotype", df, groups=df["animal"])
    mixed_lm_fit = mixed_lm.fit()
    #print(mixed_lm_fit.summary())
    
    # Make Series for measurement and associated 
    mixed_lm_multitest = statsmodels.stats.multitest.multipletests(mixed_lm_fit.pvalues)
    measurement.append(i)
    measurement_p_val.append(mixed_lm_multitest[1][1])
    #print('', mixed_lm_multitest)
    
# Make dataframe
stats_df = pd.DataFrame()
stats_df['Measurement'] = measurement
stats_df['P_Value'] = measurement_p_val

C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\micha\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the paramete

In [49]:
stats_df.head()

,Measurement,P_Value
0,Temp,0.971138
1,O2,0.199229
2,CO2,0.361083
3,dO2,0.300132
4,dCO2,0.381675


In [25]:
# Holm-Sidak adjusted P-value
hs_adj_pvalue = mixed_lm_multitest[-2]
hs_adj_pvalue

0.012741455098566168

In [51]:
stats.ttest_ind(cre[cre['time_num'] == 1]['RER'], wt[wt['time_num'] == 1]['RER'])[1]

0.6923209128871819

In [94]:
cre = df[df['Genotype'] == 'cre']
wt = df[df['Genotype'] == 'WT']

d = dict()
for i in df.columns[7:-3]:
    l = []
    for j in df['time_num'].unique():
        if stats.ttest_ind(cre[cre['time_num'] == j][i], wt[wt['time_num'] == j][i])[1] <= hs_adj_pvalue:
            l.append(j)
            d[i] = l

In [95]:
stats_df['Sig_Time_Points'] = stats_df['Measurement'].map(d)
stats_df.head()

,Measurement,P_Value,Sig_Time_Points
0,Temp,0.971138,NaN
1,O2,0.199229,"[41, 62, 92, 212, 239, 284]"
2,CO2,0.361083,"[41, 62, 92, 209, 239, 284]"
3,dO2,0.300132,"[41, 62, 92, 212, 239, 284]"
4,dCO2,0.381675,"[41, 62, 92, 209, 239, 284]"


In [97]:
stats_output = str(input('Stats Output File Name (*.csv): '))
#stats_df.to_csv(stats_output, index=False)

Stats Output File Name (*.csv):  Chow_Cohort_Ghrelin_Stats.csv
